---

# 필요 라이브러리 임포트

In [39]:
from dotenv import load_dotenv

load_dotenv()

True

---

# 논문 전처리

In [5]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

pdf_metadata = {
    "KCI_FI001522808.pdf": {
        "title": "농어촌 영아의 보육시설 이용에 영향을 미치는 요인",
        "id": "KCI_FI001522808 논문",
        "category": "농어촌, 영유아, 보육"
    },
    "KCI_FI001674019.pdf": {
        "title": "유아교육 선진화 정책의 성과와 과제",
        "id": "KCI_FI001674019 논문",
        "category": "유아교육, 정책"
    },
    "KCI_FI001674044.pdf": {
        "title": "어린이집 영유아의 보호권 보장 실태와 개선 과제",
        "id": "KCI_FI001674044 논문",
        "category": "어린이집, 유아교육, 정책"
    },
    "KCI_FI001780615.pdf": {
        "title": "취업모와 비취업모 유아의 발달에 대한 어머니의 역할, 유아 기질, 환경적 지지의 효과",
        "id": "KCI_FI001780615 논문",
        "category": "취업, 역할, 유아기질"
    },
    "KCI_FI001890689.pdf": {
        "title": "1명의 영유아 자녀를 둔 취업모의 후속출산계획에 영향을 미치는 요인",
        "id": "KCI_FI001890689 논문",
        "category": "영유아, 자녀, 취업, 후속출산계획"
    },
    "KCI_FI001952661.pdf": {
        "title": "유아의 기질 및 부모 양육방식이 유아의 또래 상호작용에 미치는 영향",
        "id": "KCI_FI001952661 논문",
        "category": "유아, 부모, 상호작용"
    },
    "KCI_FI002006648.pdf": {
        "title": "가정환경이 유아발달에 미치는 영향과 교사효능감의 조절효과",
        "id": "KCI_FI002006648 논문",
        "category": "가정환경, 유아발달, 교사효능감"
    },
    "KCI_FI002006650.pdf": {
        "title": "만 3세 유아의 기질 및 언어발달과 어머니의 자녀가치가 유아의 긍정적･부정적 또래 상호작용에 미치는 영향",
        "id": "KCI_FI002006650 논문",
        "category": "3세, 유아, 언어발달, 자녀가치, 또래, 상호작용"
    },
    "KCI_FI002073305.pdf": {
        "title": "영아기 자녀를 둔 어머니의 양육 스트레스와 긍정적 양육행동간의 관계에 대한 자아존중감의 조절효과",
        "id": "KCI_FI002073305 논문",
        "category": "영아기, 자녀, 양육, 스트레스, 자아존중감"
    },
    "KCI_FI002269551.pdf": {
        "title": "유아의 학교준비도 영향요인 분석",
        "id": "KCI_FI002269551 논문",
        "category": "유아, 학교, 영향"
    },
    "KCI_FI002355594.pdf": {
        "title": "유아의 문제행동 양상에 따른 잠재계층분류와 영향요인 검증",
        "id": "KCI_FI002355594 논문",
        "category": "유아, 문제행동, 잠재계층"
    },
    "KCI_FI002504597.pdf": {
        "title": "어머니의 출산 후 6년간 우울증상 궤적과 초등학교 1학년 아동의 정서통제 곤란",
        "id": "KCI_FI002504597 논문",
        "category": "출산, 우울증, 정서통제, 초등학교"
    },
    "KCI_FI002596727.pdf": {
        "title": "아버지의 양육참여와 양육책임감에 영향을 미치는 요인",
        "id": "KCI_FI002596727 논문",
        "category": "아버지, 양육, 참여, 책임감"
    },
    "KCI_FI002627676.pdf": {
        "title": "유아기 자녀를 둔 부모의 훈육유형 척도개발 및 타당화",
        "id": "KCI_FI002627676 논문",
        "category": "유아기, 자녀, 부모, 훈육"
    },
    "KCI_FI002664160.pdf": {
        "title": "무자녀와 한자녀 여성의 출산계획유형에 영향을 미치는 요인:자녀가치관과 부부평등성 요인을 중심으로",
        "id": "KCI_FI002664160 논문",
        "category": "무자녀, 한자녀, 여성, 출산계획, 자녀가치관, 부부평등성"
    },
    "KCI_FI002757004.pdf": {
        "title": "유아의 인성과 유아교육기관적응 간의 관계에서 의사소통능력의 매개효과",
        "id": "KCI_FI002757004 논문",
        "category": "유아, 인성, 유아교육기관, 적응, 의사소통"
    },
    "KCI_FI002881829.pdf": {
        "title": "어머니의 정서사회화 유형에 따른 유아 정서조절의 차이",
        "id": "KCI_FI002881829 논문",
        "category": "어머니, 정서사회화, 유아, 정서조절"
    },
    "KCI_FI002881834.pdf": {
        "title": "남녀 유아의 또래놀이상호작용과 사회적 유능성의 관계에서 언어발달의 매개효과",
        "id": "KCI_FI002881834 논문",
        "category": "남녀, 유아, 또래놀이, 상호작용, 언어발달"
    },
    "KCI_FI002907504.pdf": {
        "title": "영유아기 부모의 부부갈등이 초기 청소년기 자녀의 미디어기기 중독에 미치는 종단적 영향: 부모의 온정적 양육행동과 아동의 내재화 문제의 이중매개효과",
        "id": "KCI_FI002907504 논문",
        "category": "영유아기, 부모, 부부갈등, 청소년기, 자녀, 미디어기기, 아동, 양육행동"
    },
    "KCI_FI002969707.pdf": {
        "title": "영유아기 아버지의 직장 내 가족친화제도 이용이 초기 청소년기 집행기능에 미치는 영향:아동기 부모 공동양육 수행의 매개효과",
        "id": "KCI_FI002969707 논문",
        "category": "영유아기, 아버지, 직장, 가족친화제도, 청소년기, 공동양육, 아동기"
    },
    "KCI_FI002999540.pdf": {
        "title": "유아의 언어발달과 위축 및 공격성 간의 관계:어머니의 온정적 양육태도의 조절효과",
        "id": "KCI_FI002999540 논문",
        "category": "유아, 언어발달, 위축, 공격성, 양육태도"
    },
    "KCI_FI003027595.pdf": {
        "title": "어머니의 자기성찰과 유아의 심리적 건강성 간 관계에서 부모공동양육의 매개효과",
        "id": "KCI_FI003027595 논문",
        "category": "어머니, 자기성찰, 심리, 건강성, 부모공동양육"
    },
    "KCI_FI003153416.pdf": {
        "title": "출산 전후 산모의 우울 변화궤적 유형과 양육 스트레스의 관계: 육아지원정책의 조절효과",
        "id": "KCI_FI003153416 논문",
        "category": "출산, 산모, 우울, 양육, 스트레스, 육아지원정책"
    }
}


load_dotenv()
emb = OpenAIEmbeddings(model="text-embedding-3-small")

semantic_splitter = SemanticChunker(
    emb,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
)

fallback_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " ", ""]
)

pdf_directory = "./pdf_files/"
all_json_objects = []

print("PDF 파일 처리를 시작합니다...")
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        if filename not in pdf_metadata:
            print(f"경고: '{filename}'에 대한 메타데이터가 정의되지 않아 건너뜁니다.")
            continue

        file_path = os.path.join(pdf_directory, filename)
        print(f"-> '{file_path}' 파일을 처리 중입니다...")

        current_metadata = pdf_metadata[filename]

        loader = PyPDFLoader(file_path)
        documents = loader.load()

        full_text = ' '.join([doc.page_content.replace('\n', ' ') for doc in documents])

        sem_chunks = semantic_splitter.split_text(full_text)
        
        final_chunks = []
        for chunk in sem_chunks:
            if len(chunk) > 1000:
                final_chunks.extend(fallback_splitter.split_text(chunk))
            else:
                final_chunks.append(chunk)

        for chunk_text in final_chunks:
            json_object = {
                "text": chunk_text,
                "metadata": current_metadata
            }
            all_json_objects.append(json_object)

output_filename = "output.jsonl"
with open(output_filename, 'w', encoding='utf-8') as f:
    for json_obj in all_json_objects:
        f.write(json.dumps(json_obj, ensure_ascii=False) + '\n')

print("-" * 30)
print(f"총 {len(all_json_objects)}개의 청크가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")

PDF 파일 처리를 시작합니다...
-> './pdf_files/KCI_FI002881834.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI001890689.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002596727.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI003027595.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI003153416.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002907504.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002269551.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002006648.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002355594.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002073305.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002504597.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI001522808.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002757004.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002999540.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002627676.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI001780615.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI001674019.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002664160.pdf' 파일을 처리 중입니다...
-> './pdf_files/KCI_FI002969707.pdf' 파일을 처리 

---

# 아이사랑 전처리

In [2]:
import json
import re

input_file = "childcare_pages_system_user_assistant_topic_system.jsonl"
output_file = "total_childcare_pages_system_user_assistant_topic_system.jsonl"

with open(input_file, "r", encoding="utf-8") as infile, \
     open(output_file, "w", encoding="utf-8") as outfile:
    
    for line in infile:
        try:
            data = json.loads(line)
            
            system_text = data.get("system", "")
            match = re.findall(r"'(.*?)'", system_text)
            title = match[0] if match else ""
            
            text = data.get("assistant", "")
            
            new_data = {
                "text": text,
                "metadata": {
                    "title": title,
                    "id": "아이사랑 사이트",
                    "category": "임신, 출산, 육아"
                }
            }

            outfile.write(json.dumps(new_data, ensure_ascii=False) + "\n")
        
        except json.JSONDecodeError:
            continue

print("✅ 전처리 완료! 결과 파일:", output_file)

✅ 전처리 완료! 결과 파일: total_childcare_pages_system_user_assistant_topic_system.jsonl


---

# 전체 JSON 형식 데이터 전처리 (필요시 사용)

In [ ]:
import json
import re

input_file_path = '/content/drive/MyDrive/SKN17_3rd_Project/rag_output_csj.jsonl'
output_file_path = '/content/drive/MyDrive/SKN17_3rd_Project/rag_output_csj_corrected.jsonl'

try:
    with open(input_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    decoder = json.JSONDecoder()
    objects = []
    pos = 0

    NOT_WHITESPACE = re.compile(r'\S')
    while True:
        match = NOT_WHITESPACE.search(content, pos)
        if not match:
            break
        pos = match.start()
        
        try:
            obj, pos = decoder.raw_decode(content, pos)
            objects.append(obj)
        except json.JSONDecodeError:
            print(f"🚨 파싱 오류: 위치 {pos}에서 유효한 JSON 객체를 찾지 못했습니다. 파일이 심하게 손상되었을 수 있습니다.")
            break

    if objects:
        with open(output_file_path, 'w', encoding='utf-8') as f:
            for item in objects:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        
        print(f"✅ 복구 및 변환 완료! 총 {len(objects)}개의 항목이 '{output_file_path}' 파일에 저장되었습니다.")
        print("이제 기존 코드에서 파일 경로를 새 파일 이름으로 변경하여 실행해 보세요.")
    else:
        print("⚠️ 파일에서 유효한 JSON 객체를 찾지 못했습니다.")


except FileNotFoundError:
    print(f"🚨 오류: '{input_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
except Exception as e:
    print(f"🚨 알 수 없는 오류가 발생했습니다: {e}")